In [12]:
# Multinomial Classification
# BMI 예제

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler       # Normalization
from sklearn.model_selection import train_test_split # train, test 분리
from sklearn.model_selection import KFold            # Cross Validation

# test data set은 우리 모델의 최종 accuracy를 측정하기 위해서 사용
# train data set은 K-Fold Cross validation을 이용해서 내부적인 평가를 진행

# Raw Data Loading
df = pd.read_csv('./data/bmi/bmi.csv', skiprows=3)
# display(df.head(),df.shape)

# 상관분석
# 종속변수(레이블)에 영향을 미치지 않는 feature(속성, 필드)를 제외하기 위해서.
# -1 ~ 1 사이의 실수값이 나와요! pandas를 이용해서 처리를 하면 되요!
# 필요치 않은 column을 제거해요!

# 결측치 처리
# NaN이 있는지를 확인
# df.isnull() : DataFrame안의 NaN에 대한 boolean mask를 생성(NaN이면 True)
# 만약 결측치가 있으면 다른 값으로 대체할 필요가 있어요! => KNN으로 처리해보아요!
# df.isnull().sum()

# 이상치(outlier)
# boxplot과 같은 graph를 이용해서 눈으로 확인!
# turkey fense나 z-score방식으로 이상치를 찾아내고 처리하면 되요!
# 우리예제는 이상치도 존재하지 않아요!

# Data Split ( Train Data와 Test Data로 나눌꺼예요! )
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height', 'weight']],df['label'], 
                 test_size=0.3, 
                 random_state=0)  # random_state는 seed의 개념과 같아요!

# Min-Max Scaler를 이용해서 정규화(Normalization)진행
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

# 혼동을 줄이기 위해 변수 삭제
del x_data_train
del x_data_test

# Tensorflow 구현
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=3))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=3))

del t_data_train
del t_data_test

# print(t_data_train_onehot)

# placeholder
X = tf.placeholder(shape=[None,2], dtype=tf.float32)
T = tf.placeholder(shape=[None,3], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([2,3]), name='weight')
b = tf.Variable(tf.random.normal([3]), name='bias')

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)  # softmax activation function

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)


# parameter
num_of_epoch = 1000
batch_size = 100

# 학습용 함수
def run_train(sess,train_x, train_t):
    print('### 학습 시작 ###')
    # 초기화
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        total_batch = int(train_x.shape[0] / batch_size)
        
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]           
            _, loss_val = sess.run([train,loss],
                                   feed_dict={X: batch_x,
                                              T: batch_t})
            
        if step % 100 == 0:
            print('Loss : {}'.format(loss_val))
    print('### 학습 종료 ###')
    
# Accuracy 측정(정확도)    
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# 학습부터 해요!
run_train(sess,x_data_train_norm, t_data_train_onehot)
# Training Data Set을 이용하여 성능평가!
print('### Training Data Set을 이용하여 성능평가! - 좋지 않아요! ###')
result = sess.run(accuracy, feed_dict={X:x_data_train_norm,
                                       T:t_data_train_onehot})
print('Accuracy : {}'.format(result))

### 학습 시작 ###
Loss : 0.9050484299659729
Loss : 0.20565372705459595
Loss : 0.16245999932289124
Loss : 0.14281007647514343
Loss : 0.1310230791568756
Loss : 0.12295784801244736
Loss : 0.11699096858501434
Loss : 0.11234201490879059
Loss : 0.10858359932899475
Loss : 0.10545965284109116
### 학습 종료 ###
### Training Data Set을 이용하여 성능평가! - 좋지 않아요! ###
Accuracy : 0.9827142953872681
